<a href="https://colab.research.google.com/github/vieri03/Prediction-of-Apple-Ripeness/blob/main/Image_Segmentation_K_means_Clustering_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import glob
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/Colab Notebooks

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks


K-means clustering, K=3

In [ ]:
path = '/content/gdrive/My Drive/Colab Notebooks/K-means-after/*.*'

filename = '/content/gdrive/My Drive/Colab Notebooks/K-means-result/conversion_rate_after_new.csv'

# open the file in the write mode
f = open(filename, 'w')

# create the csv writer
writer = csv.writer(f)
head = ["Number","Image", "Cluster 0", "Cluster 1", "Cluster 2", "Sum Cluster 0", "Sum Cluster 1", "Sum Cluster 2"]
writer.writerow(head)

image_no = 1
for file in sorted(glob.glob(path)):
  original_image = cv.imread(file)
  img =cv.cvtColor(original_image,cv.COLOR_BGR2RGB)
  vectorize = img.reshape((-1,3))

  # convert to np.float32
  vectorized = np.float32(vectorize)

  # define criteria, number of clusters(K) and apply kmeans()
  criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
  K = 3
  attempts = 10
  ret,label,center=cv.kmeans(vectorized,K,None,criteria,attempts,cv.KMEANS_PP_CENTERS)

  # Now convert back into uint8, and make original image
  center = np.uint8(center)
  res = center[label.flatten()]
  result_image = res.reshape((img.shape))
  result_image2 = cv.cvtColor(result_image,cv.COLOR_RGB2BGR)

  result = '/content/gdrive/My Drive/Colab Notebooks/K-means-result/segmented' + str(image_no) + '.jpg'
  cv.imwrite(result, result_image2)

  pixel_0 = np.sum(center[0])
  pixel_1 = np.sum(center[1])
  pixel_2 = np.sum(center[2])

  row = [image_no, file, center[0], center[1], center[2], pixel_0, pixel_1, pixel_2]

# write a row to the csv file
  writer.writerow(row)

  image_no += 1

# close the file
f.close()

K-means clustering, K=2, RGB to Lab

In [ ]:
# RGB to Lab conversion

# Step 1: RGB to XYZ      http://www.easyrgb.com/index.php?X=MATH&H=02#text2
# Step 2: XYZ to Lab      http://www.easyrgb.com/index.php?X=MATH&H=07#text7

def rgb2lab(inputColor):

    num = 0
    RGB = [0, 0, 0]

    for value in inputColor:
        value = float(value) / 255

        if value > 0.04045:
            value = ((value + 0.055) / 1.055) ** 2.4
        else:
            value = value / 12.92

        RGB[num] = value * 100
        num = num + 1

    XYZ = [0, 0, 0, ]

    X = RGB[0] * 0.4124 + RGB[1] * 0.3576 + RGB[2] * 0.1805
    Y = RGB[0] * 0.2126 + RGB[1] * 0.7152 + RGB[2] * 0.0722
    Z = RGB[0] * 0.0193 + RGB[1] * 0.1192 + RGB[2] * 0.9505
    XYZ[0] = round(X, 4)
    XYZ[1] = round(Y, 4)
    XYZ[2] = round(Z, 4)

    # Observer= 2°, Illuminant= D65
    XYZ[0] = float(XYZ[0]) / 95.047         # ref_X =  95.047
    XYZ[1] = float(XYZ[1]) / 100.0          # ref_Y = 100.000
    XYZ[2] = float(XYZ[2]) / 108.883        # ref_Z = 108.883

    num = 0
    for value in XYZ:

        if value > 0.008856:
            value = value ** (0.3333333333333333)
        else:
            value = (7.787 * value) + (16 / 116)

        XYZ[num] = value
        num = num + 1

    Lab = [0, 0, 0]

    L = (116 * XYZ[1]) - 16
    a = 500 * (XYZ[0] - XYZ[1])
    b = 200 * (XYZ[1] - XYZ[2])

    Lab[0] = round(L, 4)
    Lab[1] = round(a, 4)
    Lab[2] = round(b, 4)

    return Lab

Source: https://gist.github.com/manojpandey/f5ece715132c572c80421febebaf66ae

Reference: https://en.wikipedia.org/wiki/CIELAB_color_space

 "CIE 1976 L*a*b* colour space | eilv". eilv.cie.co.at. Archived from the original on 2019-12-28. Retrieved 2020-12-12.

In [ ]:
path = '/content/gdrive/My Drive/Colab Notebooks/K-means/*.*'

filename = '/content/gdrive/My Drive/Colab Notebooks/K-means-result/conversion_rate_lab.csv'

# open the file in the write mode
f = open(filename, 'w')

# create the csv writer
writer = csv.writer(f)
head = ["Number","Image", "Cluster 0 RGB", "Cluster 1 RGB", "0 L", "0 a*", "0 b*", "1 L", "1 a*", "1 b*", "Sum Cluster 0", "Sum Cluster 1"]
writer.writerow(head)

image_no = 1
for file in sorted(glob.glob(path)):
  original_image = cv.imread(file)
  img =cv.cvtColor(original_image,cv.COLOR_BGR2RGB)
  vectorize = img.reshape((-1,3))

  # convert to np.float32
  vectorized = np.float32(vectorize)

  # define criteria, number of clusters(K) and apply kmeans()
  criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
  K = 2
  attempts = 10
  ret,label,center=cv.kmeans(vectorized,K,None,criteria,attempts,cv.KMEANS_PP_CENTERS)

  # Now convert back into uint8, and make original image
  center = np.uint8(center)
  res = center[label.flatten()]
  result_image = res.reshape((img.shape))
  result_image2 = cv.cvtColor(result_image,cv.COLOR_RGB2BGR)
  
  result = '/content/gdrive/My Drive/Colab Notebooks/K-means-result/segmented' + str(image_no) + '.jpg'
  cv.imwrite(result, result_image2)

  pixel_0 = np.sum(center[0])
  pixel_1 = np.sum(center[1])

  pixlab_0 = rgb2lab(center[0])
  pixlab_1 = rgb2lab(center[1])

  row = [image_no, file, center[0], center[1], pixlab_0[0], pixlab_0[1], pixlab_0[2], pixlab_1[0], pixlab_1[1], pixlab_1[2], pixel_0, pixel_1]

  # write a row to the csv file
  writer.writerow(row)

  image_no += 1

# close the file
f.close()